## 1. Import Modules

In [5]:
import pandas as pd
import numpy as np
import math
import sklearn
import matplotlib.pyplot as plt
import sqlalchemy
import pyodbc
import urllib
import time
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import gensim
import random


from gensim.models import Word2Vec, LdaModel, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from tqdm import * 

from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE # visualize high-dimensional data in a low dimension (usually 2

import matplotlib.pyplot as plt
import matplotlib.cm as cm



## 2. Define Functions

In [6]:
### Get data from sql
def GetDataFromSQL(SQL,SQLServer):
    conn , DF = None, None
    try:
        conn = pyodbc.connect(SQLServer, autocommit=True)
        DF = pd.io.sql.read_sql(SQL, conn)
    except Exception as ex:
        print(ex, "\n")
    finally:
        if conn is not None:
            conn.close()
    return DF

In [7]:
# define a class of migration functions
class Customer_Branch_Migration:
    
    def __init__(self, df):
        self.df = df
        self.from_bcn = df[df['Branch_Label'] == 'AM']['BCN'].unique()
        self.to_bcn = df[df['Branch_Label'] == 'AS']['BCN'].unique()
    
    def data_transform(self):
        # remove spaces
        self.df.CUSTNAME = self.df.CUSTNAME.apply(lambda x: x.strip())
        self.df.PRODUCTDESC1 = self.df.PRODUCTDESC1.apply(lambda x: x.strip())
        
        # create numeric id for customer + product
        self.df.BCN = self.df.BCN.astype('category')
        self.df['customer_id'] = self.df.BCN.cat.codes

        self.df.PRODUCTDESC1 = self.df.PRODUCTDESC1.astype('category')
        self.df['product_id'] = self.df.PRODUCTDESC1.cat.codes
        
    def customer2vec(self, embedding_size = 50, neighborhood_size = 100):
        n_customer = len(self.df.customer_id.unique())
        self.prod_list = [list(self.df.PRODUCTDESC1[self.df.customer_id == i]) 
                          for i in tqdm(range(n_customer))]
        
        # fit word2vec model
        self.model = Word2Vec(self.prod_list, 
                              vector_size = embedding_size,                               
                              window = neighborhood_size,
                              min_count = 1,
                              sg = 1,
                              hs = 1, 
                              negative = 5,
                              workers = 4)
        
        # get customer embeddings by averaging product representations
        self.customer_embeddings, self.bcn = [], []

        for i in range(n_customer):
            self.bcn.append(self.df.BCN[self.df.customer_id == i].iloc[0])
            self.customer_embeddings.append(self.model.wv[self.prod_list[i]].mean(axis = 0))
        
        self.customer_embeddings = pd.DataFrame(self.customer_embeddings, index = self.bcn)  

        self.customer_similarity_from = cosine_similarity(self.customer_embeddings.loc[self.from_bcn].values)
        self.customer_similarity_to = cosine_similarity(self.customer_embeddings.loc[self.to_bcn].values)
        
        self.customer_similarity_from = pd.DataFrame(self.customer_similarity_from, index = self.from_bcn, columns = self.from_bcn)
        self.customer_similarity_to = pd.DataFrame(self.customer_similarity_to, index = self.to_bcn, columns = self.to_bcn)
        
        
        
        return self.customer_embeddings
    
    def similarity_between_branches(self, embed_A, embed_B):

        # calculate similarities between current branch and another branch
        similarity_with_another_branch = cosine_similarity(embed_A.values, embed_B.values)
  
        similarity_with_another_branch_mean = similarity_with_another_branch.mean(axis = 1, keepdims = True)
        similarity_with_another_branch_mean = pd.DataFrame(similarity_with_another_branch_mean, 
                                                           index = embed_A.index,
                                                           columns = ['mean_similarity_with_another_branch'])
        similarity_with_another_branch_mean.sort_values(by = 'mean_similarity_with_another_branch', 
                                                        ascending = False, 
                                                        inplace = True)
        return similarity_with_another_branch_mean
    
    def most_similar_customers(self, bcn, topk, branch = 'from',in_branch = True):  
        if text.lower(branch) == 'from':
            target = self.customer_similarity_from
            compare_to = self.customer_similarity_to
            bcn_list = self.from_bcn
        elif text.lower(branch) == 'to':
            target = self.customer_similarity_to
            compare_to = self.customer_similarity_from
            bcn_list = self.to_bcn
        else:
            raise Exception("Choose input: 'from'/'to'")
            
        # identify similar customers within the same branch
        if in_branch: 
            bcn_similarity = target[bcn].values # similarity with other customers in current branch
            idx = bcn_similarity.argsort()[-(1 + topk):-1]
            return [(bcn_list[i], bcn_similarity[i]) for i in idx[::-1]]
        else:
        # identify similar customers from another branch
            bcn_embedding = self.customer_embeddings.loc[bcn].values
            bcn_embedding = bcn_embedding[None, :]
            bcn_similarity_with_another_branch = cosine_similarity(bcn_embedding, 
                                                                   compare_to.values)
            idx = bcn_similarity_with_another_branch.argsort()[0][-topk:]
        return [(compare_to.index[i], bcn_similarity_with_another_branch[0][i]) 
                for i in idx[::-1]]   
    
#    def product_recommendation(self, bcn, topk):
#        # personalized recommendation for customers within current branch
#        cust_prod_seq = self.prod_list[self.bcn.index(bcn)]
#        cust_invoicedt = list(self.df.INVOICEDT[self.df.BCN == bcn])
#
#        max_t = max(cust_invoicedt)
#        days_back = [(max_t - date).days for date in cust_invoicedt]
#
#        rec = []
#
#        for prod, daysback in zip(cust_prod_seq, days_back):
#
#            ms = self.model.wv.most_similar(positive = prod, topn = 10)
#            decay = 0.95**daysback
#            ms = [(i[0], decay*i[1]) for i in ms]
#
#            rec.extend(ms)
#
#        rec = [(i[0], i[1]) for i in rec if i[0] not in set(cust_prod_seq)] 
#        rec.sort(key = lambda x: -x[1])
#        
#        return rec[:topk]       

## 3. Get Data from SQL 

## Change value below:

In [8]:
#AM_table = 'dev_upload.dbo.diamond_minder_Meriplex_EU_non_msft'
#AS_table = 'dev_upload.dbo.diamond_miner_Meriplex_EU_msft'


AM_table = 'dev_upload.dbo.diamond_miner_All_EU'
AS_table = 'dev_upload.dbo.diamond_miner_Meriplex_EU'

look_back_months = '12'

In [14]:
data_query = '''
set nocount on

drop table if EXISTS #from_purchases
select distinct a.NEW_DUNSNBR_ENDUSER BCN, eu.BUSINESSNAME CUSTNAME, pr.PRODUCTKEY, pr.PRODUCTDESC1, a.INVOICEDT 
into #from_purchases
FROM BIC_DW.dbo.HI_INVOICELINE a
join BIC_DW.dbo.CS_CUSTOMER b on a.BRANCHCUSTOMERNBR = b.BRANCHCUSTOMERNBR and a.COMPANYCD = b.COMPANYCD
join bic_dw.dbo.PR_PRODUCT pr on a.PRODUCTKEY = pr.PRODUCTKEY and a.COMPANYCD = pr.COMPANYCD
JOIN py_input_table d on d.companycd = a.COMPANYCD and d.NEW_DUNSNBR_ENDUSER = a.NEW_DUNSNBR_ENDUSER  and d.Source = 'IMP'
left join bic_dw.dbo.DIMENDUSER_GLOBAL eu on eu.DUNS_NBR = a.DUNS_NBR
where a.INVOICEDT >= dateadd(year,-1,getdate())
and b.REMOVEDDT is null and b.INCARNNBR = 1
and pr.REMOVEDDT IS NULL and pr.INCARNNBR = 1


drop TABLE if exists #cmp_purchases
select c.DUNS_NBR as BCN, c.company_name as CUSTNAME, o.source_key as PRODUCTKEY , o.offer_name as PRODUCTDESC1, fs.report_date as INVOICEDT
into #cmp_purchases
from globus.dbo.fact_sales fs 
join globus.dbo.offer o on o.offer_id = fs.offer_id
join globus.[dbo].[company_Duns] c on c.company_id = fs.customer_id and c.hierarhy_level = '3'
JOIN py_input_table d on d.NEW_DUNSNBR_ENDUSER = c.DUNS_NBR and d.source = 'CMP'
WHERE fs.marketplace_id = '2'
and fs.report_date >= dateadd(year,-1,getdate())
and c.duns_nbr <> '000000000'
group by c.DUNS_NBR, c.company_name, o.source_key , o.offer_name , fs.report_date


Drop table if exists #purchases
SELECT * --results 
INTO #purchases
FROM #from_purchases
union 
select *
from #cmp_purchases

-- remove duplicated SKUs with each customer on the same day
;WITH CTE AS
(
SELECT *, ROW_NUMBER() OVER (PARTITION BY BCN	, PRODUCTKEY, INVOICEDT ORDER BY BCN, PRODUCTKEY, INVOICEDT) AS RN
FROM #purchases
)

DELETE FROM CTE WHERE RN <> 1;

-- remove products with unknown description
DELETE FROM #purchases
where productdesc1 like '%unknown%'

-- ensure product-productdesc is 1-to-1 mapping (different products can have the same productdesc) 
drop table IF EXISTS #prod_proddesc
SELECT distinct PRODUCTDESC1, PRODUCTKEY
INTO #prod_proddesc
from #purchases

;WITH CTE AS
(
SELECT *, ROW_NUMBER() OVER (PARTITION BY PRODUCTDESC1
ORDER BY PRODUCTDESC1) AS RN
FROM #prod_proddesc
)
DELETE FROM CTE WHERE RN <> 1;

;WITH CTE AS
(
SELECT *, ROW_NUMBER() OVER (PARTITION BY PRODUCTKEY
ORDER BY PRODUCTKEY) AS RN
FROM #prod_proddesc
)
DELETE FROM CTE WHERE RN <> 1;



-- final table
drop table if EXISTS #purchases_2
SELECT a.*
into #purchases_2 
from #purchases a
join #prod_proddesc b on a.productkey = b.productkey AND a.PRODUCTDESC1 = b.PRODUCTDESC1
--JOIN #cust_custname c on a.bcn = c.bcn and a.custname = c.custname


drop table if EXISTS #from_large_purchase
SELECT BCN, CUSTNAME, count(distinct PRODUCTKEY) sku_counts
into #from_large_purchase
from #purchases_2
group by BCN, CUSTNAME
having count(distinct productkey) >= 5


SELECT a.* 
from #purchases_2 a
join #from_large_purchase b on a.BCN = b.BCN
ORDER BY BCN, INVOICEDT

'''

In [15]:
AM_query = data_query.replace("py_input_table",AM_table)
AS_query = data_query.replace("py_input_table",AS_table)

In [16]:
rev_query = '''
select b.NEW_DUNSNBR_ENDUSER BCN, sum(i.EXTENDEDSALES) rev
from BIC_REFERENCE.dbo.HI_INVOICELINE_NORM i
join py_input_table b on b.NEW_DUNSNBR_ENDUSER= i.DUNS_NBR
where i.INVOICEDT BETWEEN DATEADD(month, -'''+look_back_months+''', GETDATE()) and GETDATE()
group by b.NEW_DUNSNBR_ENDUSER
'''

In [17]:
AS_rev = rev_query.replace("py_input_table",AS_table)

In [18]:
#filtering = ' ORDER BY BCN, INVOICEDT'
df_am = GetDataFromSQL(AM_query, "DSN=1056d")

In [19]:
df_as = GetDataFromSQL(AS_query, "DSN=1056d")

In [20]:
df_as_rev = GetDataFromSQL(AS_rev, "DSN=1056d")

In [21]:
df_am['Branch_Label'] = 'AM'
df_as['Branch_Label'] = 'AS' 
df_full = pd.concat([df_am, df_as]).dropna().reset_index(drop = True)

## 4. Reseller Embeddings

Apply Word2Vec to resellers' product purchasing sequence to generate numerical vectors for each reseller. 
- AM: **am_cust_embeds**
- AS: **as_cust_embeds**

In [22]:
full = Customer_Branch_Migration(df_full)

In [23]:
full.data_transform()

In [24]:
full_embed = full.customer2vec()

100%|██████████| 30099/30099 [00:27<00:00, 1076.37it/s]


In [25]:
AM_embed = full_embed.loc[df_full[df_full['Branch_Label'] == 'AM']['BCN'].unique()]
AS_embed = full_embed.loc[df_full[df_full['Branch_Label'] == 'AS']['BCN'].unique()]

## 5. Identify AM Resellers Most Similar to AS Resellers

Two steps:
- Partition AS resellers into two groups: **GOOD** and **BAD**. Good AS resellers better represent resellers' characteristics in AS. They will be used to compare similarity with AM resellers.
- Measure the similarity between AM resellers and **GOOD** AS resellers. AM resellers with high similarities are considered as migration candidates.

### 5.1. Identify Good and Bad AS customers
For resellers in AS who are least similar to AM, they are considered as good AS resellers.

In [26]:
# compare individual AS resellers to AM branch
as_to_am_sim = full.similarity_between_branches(AS_embed, AM_embed)

In [27]:
as_to_am_sim.shape

(391, 1)

In [28]:
# use median as threshold separating good and bad AS resellers
thresh = as_to_am_sim.mean_similarity_with_another_branch.median()
thresh

0.3406428098678589

In [29]:
AM_embed.shape

(29979, 50)

In [30]:
# customers who are less likely to migrate to AM
# these are GOOD customers in AS
good_AS_cust_embed = AS_embed[as_to_am_sim.mean_similarity_with_another_branch <= thresh] 
bad_AS_cust_embed = AS_embed[as_to_am_sim.mean_similarity_with_another_branch > thresh] 

### 5.2. Identify Similarity Between AM Resellers and Good AS Resellers
For AM resellers who are very similar to good AS resellers, they will be top candidates for migration. Threshold is set at top 20%. 

In [31]:
# similarities between AM and good AS resellers
am_to_as_good_similarity = full.similarity_between_branches(AM_embed, good_AS_cust_embed)

In [32]:
am_to_as_good_similarity.index = am_to_as_good_similarity.index.set_names(['BCN'])

In [33]:
am_to_as_good_similarity.reset_index(inplace = True)

In [34]:
am_to_as_good_similarity

,BCN,mean_similarity_with_another_branch
0,010726883,0.500938
1,007436605,0.500634
2,097348114,0.499478
3,060273687,0.498949
4,066780667,0.498155
...,...,...
29974,928327063,-0.112448
29975,079417891,-0.115503
29976,176916583,-0.121083
29977,956871495,-0.127383


In [35]:
am_to_as_good_similarity.mean_similarity_with_another_branch = round(am_to_as_good_similarity.mean_similarity_with_another_branch, 4)

In [36]:
am_to_as_good_candidates = am_to_as_good_similarity[am_to_as_good_similarity.mean_similarity_with_another_branch >= am_to_as_good_similarity.mean_similarity_with_another_branch.quantile(0.7)]

### 6. Output

In [41]:
# output csv file
file_name = 'M_EU_similarity_Heartland.csv'
am_to_as_good_candidates.to_csv(file_name, index = False)

In [38]:
from gbi_utils import connector

In [39]:
#connector.upload(am_to_as_similarity, sql_table_name=file_name, server= 'USCHWSQL1056D', database= 'DEV_UPLOAD', schema_name='DBO')

In [40]:
am_to_as_good_candidates

,BCN,mean_similarity_with_another_branch
0,010726883,0.5009
1,007436605,0.5006
2,097348114,0.4995
3,060273687,0.4989
4,066780667,0.4982
...,...,...
8996,188414197,0.3305
8997,838833069,0.3305
8998,122035918,0.3305
8999,071906432,0.3305


#### 